In [6]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from collections import defaultdict
import time
import string

# BeautifulSoup Scraping
* Scraping pro-football reference for all players and associated URLs
* Taking only quaterbacks
* Use these names to find the associated college quarterback records
* Use these URLs to retrieve NFL player stats

In [281]:
dict_players = {}
dict_players['player'] = []
dict_players['url'] = []
dict_players['pos'] = []

for letter in string.ascii_uppercase:
    t0 = time.time()
    url_p = 'https://www.pro-football-reference.com/players/' + letter + '/'
    
    response_p=requests.get(url_p)
    page_p=response_p.text

    soup_p=BeautifulSoup(page_p,"lxml")

    for row in soup_p.find_all('p'):
        if row.find('a') is not None:
            if row.find('a').get_attribute_list('href')[0][:9] == '/players/':
                dict_players['player'].append(row.find('a').text)
                dict_players['url'].append(row.find('a').get_attribute_list('href')[0])
                dict_players['pos'].append((re.search(r'\((.*?)\)',row.text).group(1)))
    total = 0              
    while total < 5:
        t1 = time.time()
        total = t1-t0

In [282]:
df_players = pd.DataFrame(dict_players)
df_qb = df_players[df_players.pos == 'QB'].copy()
df_qb.reset_index(inplace=True)

In [285]:
qb_url = df_qb.url.tolist()

In [312]:
df_players.to_csv('nfl_players.csv')

# Scraping NFL QB Stats
* Using names from above scraping
* Get player NFL stats

In [8]:
df_qb = pd.read_csv('../nfl_players.csv')
df_qb = df_qb[df_qb.pos == 'QB'].reset_index()

In [ ]:
# TODO: make these nested if statements better...

dict_player_year = {}
dict_player_year = defaultdict(list)

for i in range(len(df_qb)):
    t0 = time.time()
    url_nfl = 'https://www.pro-football-reference.com' + df_qb.url[i]

    response_nfl=requests.get(url_nfl)
    page_nfl=response_nfl.text

    soup_nfl=BeautifulSoup(page_nfl,"lxml")

    for row in soup_nfl.find_all(class_='full_table'):
        if row.find('th',{'data-stat':'year_id'}).find('a') is not None:
            if row.find('td',{'data-stat':'gs'}) is not None:
                if row.find('td',{'data-stat':'pass_cmp'}) is not None:
                    if row.find('td',{'data-stat':'pass_att'}) is not None:
                        if row.find('td',{'data-stat':'pass_yds'}) is not None:
                            if row.find('td',{'data-stat':'pass_td'}) is not None:
                                if row.find('td',{'data-stat':'pass_int'}) is not None:
                                    if row.find('td',{'data-stat':'team'}) is not None:
                                        if row.find('td',{'data-stat':'g'}) is not None:
                                            player = df_qb.player[i]
                                            dict_player_year['player'].append(player)
                                            dict_player_year['year'].append(row.find('th',{'data-stat':'year_id'}).find('a').text)
                                            dict_player_year['games'].append(row.find('td',{'data-stat':'gs'}).text)
                                            dict_player_year['pass_cmp'].append(row.find('td',{'data-stat':'pass_cmp'}).text)
                                            dict_player_year['pass_att'].append(row.find('td',{'data-stat':'pass_att'}).text)
                                            dict_player_year['pass_yds'].append(row.find('td',{'data-stat':'pass_yds'}).text)
                                            dict_player_year['pass_td'].append(row.find('td',{'data-stat':'pass_td'}).text)
                                            dict_player_year['team'].append(row.find('td',{'data-stat':'team'}).text)
                                            dict_player_year['games_played'].append(row.find('td',{'data-stat':'g'}).text)

    total = 0  
    while total < 0:
        t1 = time.time()
        total = t1-t0


In [20]:
nfl_stats.to_csv('nfl_stats.csv')

# Finding CFB quarterbacks
* Getting all CFB-Reference player names & URLs
* Going to cross-reference this with NFL QB's to get final list
* Little tricky to iterate through player pages

In [ ]:
dict_players_cfb = {}
dict_players_cfb['player'] = []
dict_players_cfb['url'] = []

url_ext = ''
i = 0
letter = '`'

while letter != {:
    t0 = time.time()
    url_ext = url_ext[:7] + '-' + str(i+1) + url_ext[-5:]
    url_p = 'https://www.sports-reference.com/cfb/players/' + url_ext
    response_p=requests.get(url_p)

    if response_p.status_code != 200:
        i = 0
        letter = chr(ord(letter) + 1)
        url_ext = letter + '-index.html'
        url_p = 'https://www.sports-reference.com/cfb/players/' + url_ext
        response_p=requests.get(url_p)
    print(url_ext)
    page_p=response_p.text
    soup_p=BeautifulSoup(page_p,"lxml")

    for row in soup_p.find_all('p'):
        if row.find('a') is not None:
            if row.find('a').get_attribute_list('href')[0][:13] == '/cfb/players/':
                dict_players_cfb['player'].append(row.find('a').text)
                dict_players_cfb['url'].append(row.find('a').get_attribute_list('href')[0])
    i = i + 1
    total = 0  
    while total < 5:
        t1 = time.time()
        total = t1-t0
    

In [2]:
dict_players_cfb.to_csv('college_players.csv')

# Compare CFB & NFL Player lists
* Only keep players from both lists

In [9]:
df_players_cfb = pd.read_csv('college_players.csv')
df_players_nfl = pd.read_csv('nfl_players.csv')

In [10]:
df_nfl_qb = df_players_nfl[df_players_nfl.pos == 'QB'].copy()
df_both_qb = df_nfl_qb.merge(df_players_cfb, on='player').reset_index()

# Getting CFB Player Stats
* Using combined list of QB names
* Getting passing stats using BeautifulSoup
* Getting rushing and receiving stats using Selenium w/ BeautifulSoup
    * For some reason these tables are created separately

In [28]:
df_nfl_qb['full_url'] = df_nfl_qb.apply(lambda x: 'https://www.pro-football-reference.com' + x['url'], axis = 1)

In [3]:
url_cfb_head = 'https://www.sports-reference.com' 
dict_cfb_career = defaultdict(list)
    
for index in df_both_qb.index:
    t0 = time.time()
    url_cfb = url_cfb_head + df_both_qb.url_y[index]
    response_cfb=requests.get(url_cfb)
    page_cfb=response_cfb.text

    soup_cfb=BeautifulSoup(page_cfb,"lxml")

    player = df_both_qb.player[index]

    if soup_cfb.find('tbody') is not None:
        if soup_cfb.find('table').get_attribute_list('id')[0] == 'passing':
            tbl = soup_cfb.find('tbody')

            for row in tbl.find_all('th'):
                dict_cfb_career['player'].append(player)
                dict_cfb_career[row['data-stat']].append(row.text)

            for row in tbl.find_all('td'):
                dict_cfb_career[row['data-stat']].append(row.text)

    total = 0  
    while total < 5:
        t1 = time.time()
        total = t1-t0

In [36]:
df_qb_passing = pd.DataFrame(dict_cfb_career)
df_qb_passing.to_csv('cfb_qb_passing.csv')

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import os
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


In [4]:
url_cfb_head = 'https://www.sports-reference.com' 
dict_cfb_career_rush = defaultdict(list)
driver = webdriver.Chrome(chromedriver)

for index in df_both_qb.index:
    t0 = time.time()
    url_cfb = url_cfb_head + df_both_qb.url_y[index]

    driver.get(url_cfb)

    soup_cfb2 = BeautifulSoup(driver.page_source, 'html.parser')

    player = df_both_qb.player[index]
    
    if len(soup_cfb2.find_all('tbody')) > 2:
        if soup_cfb2.find_all('table')[2].get_attribute_list('id')[0] == 'rushing':
            tbl = soup_cfb2.find_all('tbody')[2]
            
            for row in tbl.find_all('th'):
                dict_cfb_career_rush['player'].append(player)
                dict_cfb_career_rush[row['data-stat']].append(row.text)

            for row in tbl.find_all('td'):
                dict_cfb_career_rush[row['data-stat']].append(row.text)
    total = 0  
    while total < 5:
        t1 = time.time()
        total = t1-t0
        
driver.close()

In [35]:
df_cfb_rush = pd.DataFrame(dict_cfb_career_rush)
df_cfb_rush.to_csv('cfb_qb_rushing')

# Player Bio Scraping
* Used Scrapy for learning purposes - BeautifulSoup may be easier
* See NFL_Spider.py for scrapy script